Classification of Diseased and Healthy 3D Coronary Artery Shapes Using MONAI (Minimal Reproducible Example)

installation

In [3]:
!pip install monai
!pip install MedShapeNetCore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.0/689.0 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.6 MB/s eta 0:00:00
  Created wheel for clint: filename=clint-0.5.1-py3-none-any.whl size=34457 sha256=e6e7ad3591d49b6ab17c707a360c7e8aa3927a70f4327370a3c470d4770e735c
 

download the dataset

In [4]:
!python -m MedShapeNetCore download ASOCA

downloading...
[################################] 42842/42842 - 00:00:12
download complete...
file directory: ./medshapenetcore_npz/medshapenetcore_ASOCA.npz


import necessay packages

In [5]:
import os
import sys
import monai
import torch
import numpy as np
from MedShapeNetCore.MedShapeNetCore import MyDict,MSNLoader,MSNVisualizer,MSNSaver,MSNTransformer
pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('using:',device)



using: cuda


load and prepare the shape data

In [7]:
msn_loader=MSNLoader()
ASOCA_DATA=msn_loader.load('ASOCA')
shape_data=ASOCA_DATA['mask']
shape_labels=ASOCA_DATA['labels']
print(shape_data.shape)
print(shape_labels)
labels = torch.nn.functional.one_hot(torch.as_tensor(shape_labels).to(torch.int64)).float() #  one hot encoding
print(labels.shape)

current dataset: ./medshapenetcore_npz/medshapenetcore_ASOCA.npz
available keys in the dataset: ['mask', 'point', 'mesh', 'labels']
(40, 256, 256, 256)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
torch.Size([40, 2])


define the optimizer, loss function and a classification model based on DenseNet

In [8]:
model = monai.networks.nets.DenseNet121(spatial_dims=3, in_channels=1, out_channels=2).to(device)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 1e-4)

train the model for 200 epochs

In [ ]:
max_epochs=200
torch.cuda.empty_cache()
import gc
gc.collect()
for epoch in range(max_epochs):
    print(f"epoch {epoch + 1}/{max_epochs}")
    model.train()
    epoch_loss = 0
    for i in range(len(shape_labels)):
        torch.cuda.empty_cache()
        inputs = torch.tensor(np.expand_dims(np.expand_dims(shape_data[i],axis=0),axis=0),dtype=torch.float32).to(device)
        labels = torch.tensor(np.expand_dims(shape_labels[i],axis=0),dtype=torch.float32).type(torch.LongTensor).to(device)
        torch.cuda.empty_cache()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = 20*loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        print(f"train_loss: {loss.item():.4f}")